In [57]:
import datetime
import glob
import json
import pandas as pd
import pickle
import numpy as np
import requests

# 소상공인

In [2]:
smallbiz_dates = [201812, 201903, 201906, 201909, 201912, 202003, 202006]
smallbiz_date_index = {date: i for i, date in enumerate(smallbiz_dates)}

In [3]:
lines = []
smallbiz_no = {}

for date in smallbiz_dates:
    temp_df = pd.read_pickle('외부데이터/공공데이터포털/소상공인시장진흥공단_상가(상권)정보/smallbiz_roi_df_%d.pkl' % date)
    
    for i, row in temp_df.iterrows():
        if row['상가업소번호'] not in smallbiz_no:
            lines.append(row)
            smallbiz_no[row['상가업소번호']] = [date]
        else:
            smallbiz_no[row['상가업소번호']].append(date)
            
smallbiz_df = pd.DataFrame(lines)
smallbiz_df = smallbiz_df.set_index('상가업소번호')
smallbiz_df.to_pickle('외부데이터/공공데이터포털/소상공인시장진흥공단_상가(상권)정보/smallbiz_roi_df_201812-202006.pkl')

with open('외부데이터/공공데이터포털/소상공인시장진흥공단_상가(상권)정보/smallbiz_number-to-dates_201812-202006.pkl', 'wb') as file:
    pickle.dump(file=file, obj=smallbiz_no)

In [4]:
smallbiz_df.head(1)

,상호명,지점명,상권업종대분류코드,상권업종대분류명,상권업종중분류코드,상권업종중분류명,상권업종소분류코드,상권업종소분류명,표준산업분류코드,표준산업분류명,...,건물관리번호,건물명,도로명주소,구우편번호,신우편번호,동정보,층정보,호정보,경도,위도
상가업소번호,,,,,,,,,,,,,,,,,,,,,
19977290,피자마루,약수점,Q,음식,Q07,패스트푸드,Q07A01,피자전문,I56192,"피자, 햄버거, 샌드위치 및 유사 음식점업",...,1114016200103720095000001,현진빌딩,서울특별시 중구 동호로7길 37,100450.0,4596.0,NaN,1,NaN,127.010861,37.552153


In [5]:
smallbiz_df.iloc[0]

상호명                               피자마루
지점명                                약수점
상권업종대분류코드                            Q
상권업종대분류명                            음식
상권업종중분류코드                          Q07
상권업종중분류명                         패스트푸드
상권업종소분류코드                       Q07A01
상권업종소분류명                          피자전문
표준산업분류코드                        I56192
표준산업분류명        피자, 햄버거, 샌드위치 및 유사 음식점업
시도코드                                11
시도명                              서울특별시
시군구코드                            11140
시군구명                                중구
행정동코드                       1114063500
행정동명                               약수동
법정동코드                       1114016200
법정동명                               신당동
지번코드                       1.11402e+18
대지구분코드                               1
대지구분명                               대지
지번본번지                              372
지번부번지                               95
지번주소               서울특별시 중구 신당동 372-95
도로명코드                      1.11404e+11
도로명                     서

In [24]:
lines = []

for index, row in smallbiz_df.iterrows():
    if pd.notnull(row['지점명']):
        name = '%s %s' % (row['상호명'], row['지점명'])
    else:
        name = row['상호명']
        
    start_date = min(smallbiz_no[index])
    end_date = max(smallbiz_no[index])
    
    fail = (end_date != 202006)
    
    if gaps and max(gaps) > 1:
        print(name, smallbiz_no[index])
        
    line = [name, row['시도명'], row['시군구명'], row['행정동명'], 
            row['상권업종대분류명'], row['상권업종중분류명'], row['상권업종소분류명'], 
            start_date, end_date, fail, row['층정보'], index]
    lines.append(line)

In [25]:
preproc_smallbiz_df = pd.DataFrame(lines, columns=['상호', '시도', '시군구', '행정동', '대분류', '중분류', '소분류', 'start_date', 'end_date', 'fail', '층정보', 'index'])
preproc_smallbiz_df.to_pickle('외부데이터/공공데이터포털/소상공인시장진흥공단_상가(상권)정보/preproc_smallbiz_roi_201812-202006.pkl')

In [174]:
len(preproc_smallbiz_df)

76413

In [30]:
preproc_smallbiz_df[preproc_smallbiz_df['fail'] == True][::200]

,상호,시도,시군구,행정동,대분류,중분류,소분류,start_date,end_date,fail,층정보,index
21,화성진열장,서울특별시,중구,광희동,소매,철물/난방/건설자재소매,진열장판매,201812,202003,True,1,22318339
2944,양지아트제본,서울특별시,중구,광희동,생활서비스,대행업,복사대행,201812,202003,True,NaN,22268238
6064,상록수약국,서울특별시,노원구,중계1동,의료,약국/한약방,약국,201812,201912,True,1,20914643
8628,주부곱창,서울특별시,중구,신당동,음식,한식,한식/백반/한정식,201812,202003,True,1,26392430
12135,대한가축병원,서울특별시,중구,광희동,의료,수의업,동물병원,201812,201912,True,NaN,21837166
14206,고은소아과,서울특별시,노원구,하계1동,의료,병원,내과/외과,201812,201912,True,3,20893545
14916,김순화소아과,서울특별시,중구,약수동,의료,병원,소아과,201812,201912,True,NaN,23175982
15614,명성한의원,서울특별시,노원구,공릉1동,의료,병원,한의원,201812,201912,True,2,22996965
16414,아가사랑어린이집,서울특별시,노원구,공릉1동,학문/교육,유아교육,어린이집,201812,202003,True,1,27794051
18023,대웅입시학원,서울특별시,노원구,중계1동,학문/교육,학원-보습교습입시,학원-입시,201812,202003,True,9,25873240


# 소상공인 & 인허가

In [23]:
data_dir = '외부데이터/지방행정 인허가 데이터/20200831/'

In [34]:
def get_road_address(row):
    # 도로명 주소 결과가 있고 0이 아님
    if pd.notnull(row['도로명주소검색결과']) and row['도로명주소검색결과']['meta']['totalCount'] > 0:
        item = row['도로명주소검색결과']['addresses'][0]['roadAddress']
        return item
    
    # 지번 주소 결과가 있고 0이 아님
    elif pd.notnull(row['지번주소검색결과']) and row['지번주소검색결과']['meta']['totalCount'] > 0:
        item = row['지번주소검색결과']['addresses'][0]['roadAddress']
        return item
    # 결과가 0개인 경우
    else:
        return np.nan

In [31]:
def get_gps_x_y(row):
    # 도로명 주소 결과가 있고 0이 아님
    if pd.notnull(row['도로명주소검색결과']) and row['도로명주소검색결과']['meta']['totalCount'] > 0:
        item = row['도로명주소검색결과']['addresses'][0]
        return float(item['x']), float(item['y'])
    
    # 지번 주소 결과가 있고 0이 아님
    elif pd.notnull(row['지번주소검색결과']) and row['지번주소검색결과']['meta']['totalCount'] > 0:
        item = row['지번주소검색결과']['addresses'][0]
        return float(item['x']), float(item['y'])
    # 결과가 0개인 경우
    else:
        return 0, 0

In [32]:
def get_end_date(row):
    if pd.notnull(row['폐업일']):
        return int(row['폐업일']), False
    if pd.notnull(row['휴업시작일']) and row['영업상태'] == '휴업':
        return int(row['휴업시작일']), False
    if row['영업상태'] in {'폐업', '휴업', '취소/말소/만료/정지/중지'}:
        return int(str(row['데이터']['최종수정시점'])[:8]), True
    if row['영업상태'] == '영업/정상':
        return np.nan, False
    raise ValueError()

In [35]:
df = pd.read_pickle('외부데이터/지방행정 인허가 데이터/roi_20200922.pkl')
df['end_date'], df['censored'] = zip(*[get_end_date(row) for i, row in df.iterrows()])
df['도로명주소'] = [get_road_address(row) for i, row in df.iterrows()]
df['x'], df['y'] = zip(*[get_gps_x_y(row) for i, row in df.iterrows()])

In [36]:
print(len(df))
df = df[df['end_date'].fillna(20200831) >= 20180901]
df = df[df['지역'].isin({'서울특별시 중구', '서울특별시 노원구', '대구광역시 수성구', '대구광역시 중구'})]
df = df[df['업종'] != '담배소매업']
print(len(df))

241494
110572


In [49]:
df.head(1)

,업종,상세업종,사업장명,영업상태,상세영업상태,인허가일,폐업일,휴업시작일,휴업종료일,재개업일,데이터,지번주소검색결과,도로명주소검색결과,행정동,지역,end_date,censored,도로명주소,x,y
0,병원,한방병원,광화문자생한방병원,영업/정상,영업중,20160708,NaN,NaN,NaN,NaN,"{'번호': 14, '개방서비스명': '병원', '개방서비스ID': '01_01_0...","{'status': 'OK', 'meta': {'totalCount': 1, 'pa...","{'status': 'OK', 'meta': {'totalCount': 1, 'pa...",소공동,서울특별시 중구,NaN,False,서울특별시 중구 통일로 92 케이지타워,126.969239,37.563598


In [183]:
df.iloc[0]['데이터']

{'번호': 14,
 '개방서비스명': '병원',
 '개방서비스ID': '01_01_01_P',
 '개방자치단체코드': 3010000,
 '관리번호': 'PHMA220163010033021200001',
 '인허가일자': 20160708,
 '인허가취소일자': nan,
 '영업상태구분코드': 1,
 '영업상태명': '영업/정상',
 '상세영업상태코드': '13',
 '상세영업상태명': '영업중',
 '폐업일자': nan,
 '휴업시작일자': nan,
 '휴업종료일자': nan,
 '재개업일자': nan,
 '소재지전화': '02-2165-0965',
 '소재지면적': nan,
 '소재지우편번호': nan,
 '소재지전체주소': '서울특별시 중구 순화동 1번지 170호 케이지타워',
 '도로명전체주소': '서울특별시 중구 통일로 92, 케이지타워 1,2,8층 (순화동)',
 '도로명우편번호': 4517.0,
 '사업장명': '광화문자생한방병원',
 '최종수정시점': 20190801154006,
 '데이터갱신구분': 'U',
 '데이터갱신일자': '2019-08-03 02:40:00.0',
 '업태구분명': '한방병원',
 '좌표정보(X)': 197214.205778633,
 '좌표정보(Y)': 451259.671938774,
 '의료기관종별명': '한방병원',
 '의료인수': 21.0,
 '입원실수': 10.0,
 '병상수': 50.0,
 '총면적': 2454.42,
 '진료과목내용': '308 307 306 305 304 303 302 301 116 101',
 '진료과목내용명': '내과, 영상의학과, 한방내과, 한방부인과, 한방소아과, 한방안·이비인후·피부과, 한방신경정신과, 한방재활의학과, 사상체질과, 침구과',
 '지정취소일자': nan,
 '완화의료지정형태': nan,
 '완화의료담당부서명': nan,
 '구급차특수': 0.0,
 '구급차일반': 0.0,
 '총인원': nan,
 '구조사수': nan,
 '허가병상수': 0.0,
 '최초지정일자': na

In [37]:
indices = []

for i, (idx, row) in enumerate(smallbiz_df.iterrows()):
    result = df[df['사업장명'].str.contains(row['상호명'].replace('?', '')) & 
                (df['도로명주소'].str.contains(row['도로명주소']))]

    if len(result):
        indices.append(tuple(result.index))
    else:
        indices.append(np.nan)
        
    if i % 1000 == 0:
        print(i)

C:\Users\eyshi\Anaconda3\lib\site-packages\pandas\core\strings.py:1954: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)


In [38]:
indices

[(119144,),
 nan,
 (134435,),
 (101008,),
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 (182432,),
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 (116281,),
 nan,
 nan,
 (133055,),
 nan,
 nan,
 (121440,),
 nan,
 nan,
 nan,
 nan,
 nan,
 (117521,),
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 (177135,),
 nan,
 nan,
 nan,
 (100997,),
 (56864, 183025),
 (118330,),
 (115634,),
 (56741, 155381),
 (58103,),
 (115838, 157707),
 (117752,),
 (117183,),
 (119981,),
 (134393,),
 (88980,),
 nan,
 (156575, 161188),
 nan,
 (88214,),
 nan,
 nan,
 (88229,),
 nan,
 nan,
 (117729,),
 nan,
 (133394,),
 nan,
 (133392,),
 (88230,),
 nan,
 (99680,),
 (65455, 115434),
 nan,
 nan,
 (132243,),
 (130906,),
 nan,
 (3678, 10504, 56378, 156546, 232317),
 nan,
 nan,
 (45055,),
 nan,
 nan,
 nan,
 nan,
 nan,
 (135158,),
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,


In [45]:
union_index = {}

for so_index, in_index in zip(smallbiz_df.index, indices):
    if pd.notnull(in_index):
        union_index[so_index] = in_index

In [46]:
with open('20200923_union_index.pkl', 'wb') as file:
    pickle.dump(file=file, obj=union_index)

In [51]:
with open('20200923_union_index.pkl', 'rb') as file:
    union_index = pickle.load(file=file)

In [48]:
len(union_index)

16834

In [88]:
in_item

업종                                                       통신판매업
상세업종                                               의류/패션/잡화/뷰티
사업장명                                                       라메르
영업상태                                                     영업/정상
상세영업상태                                                    정상영업
인허가일                                               2.02007e+07
폐업일                                                        NaN
휴업시작일                                                      NaN
휴업종료일                                                      NaN
재개업일                                                       NaN
데이터          {'번호': 410373, '개방서비스명': '통신판매업', '개방서비스ID': '...
지번주소검색결과     {'status': 'OK', 'meta': {'totalCount': 1, 'pa...
도로명주소검색결과    {'status': 'OK', 'meta': {'totalCount': 0, 'co...
행정동                                                       성내1동
지역                                                    대구광역시 중구
end_date                                               

In [90]:
in_status = {
    '영업/정상': 0,
    '폐업': 1,
    '휴업': 1,
    '취소/말소/만료/정지/중지': 2
}

In [97]:
area_list = ['총면적', '약국영업면적', '시설면적', '시설총규모', '건축물연면적']
money_list = ['자본금', '보증액', '월세액']
employee_list = ['총인원', '종업원수', '총종업원수']

In [98]:
lines = []

for so_index, in_indices in union_index.items():
    so_item = smallbiz_df.loc[so_index]
    in_items = [df.loc[in_index] for in_index in in_indices]
    
    if pd.notnull(so_item['지점명']):
        name = '%s %s' % (so_item['상호명'], so_item['지점명'])
    else:
        name = so_item['상호명']
    
    start_dates = np.array([float(item['인허가일']) for item in in_items])
    end_dates = np.array([item['end_date'] for item in in_items])
    
    if pd.isnull(start_dates).all():
        # 인허가일 없음 = 안 쓰는 데이터
        continue
    else:
        start_date = int(min(start_dates))
        in_item = in_items[np.argmin(start_dates)]
    
    if pd.isnull(end_dates).any():
        end_date = 20200831
        fail = False
        is_accurate_end_date = True
    else:
        end_date = int(max(end_dates))
        fail = True
        is_accurate_end_date = not in_items[np.argmax(end_dates)]['censored']
        
    duration = (datetime.datetime.strptime(str(end_date), '%Y%m%d') - datetime.datetime.strptime(str(start_date), '%Y%m%d')).days
    
    if in_item['영업상태'] == '휴업' or pd.notnull('휴업시작일자') or pd.notnull('휴업종료일자'):
        have_been_rest = True
    else:
        have_been_rest = False
        
    in_categories = set([(in_item['업종']) for in_item in in_items])
    
    can_sell_internet = '통신판매업' in in_categories
    can_sell_cigar = '담배소매업' in in_categories
    can_sell_can = '식품자동판매기업' in in_categories
    
    in_categories = [(in_item['업종'], in_item['상세업종']) for in_item in in_items]
    
    area = np.nan
    for area_name in area_list:
        if area_name in in_item:
            area = in_item[area_name]
            break
    
    employee = np.nan
    for employee_name in employee_list:
        if employee_name in in_item:
            employee = in_item[employee_name]
            break
        
    capital = in_item.get('자본금', np.nan)
    deposit = in_item.get('보증금', np.nan)
    rent = in_item.get('월세', np.nan)
        
    line = [name, so_item['시도명'], so_item['시군구명'], so_item['행정동명'], 
            so_item['상권업종대분류명'], so_item['상권업종중분류명'], so_item['상권업종소분류명'], 
            start_date, end_date, duration, fail, so_item['층정보'],
            in_status[in_item['영업상태']], have_been_rest, is_accurate_end_date, 
            can_sell_internet, can_sell_cigar, can_sell_can, in_categories,
            area, employee, capital, deposit, rent, 
            so_index, in_indices]
    lines.append(line)

In [100]:
preproc_union_df = pd.DataFrame(
    lines, 
    columns=['상호', '시도', '시군구', '행정동', '대분류', '중분류', '소분류', 'start_date', 'end_date', 'duration', 'fail', '층정보', 
             '영업상태', 'have_been_휴업', 'is_accurate_end_date', '통신판매', '담배소매', '자동판매기', '인허가목록',
             '면적', '종업원수', '자본금', '보증금', '월세', '소상공인_index', '인허가_index'])

In [101]:
preproc_union_df.to_pickle('외부데이터/지방행정 인허가 데이터/preproc_approval_roi_df.pkl')

In [103]:
preproc_union_df[preproc_union_df['fail'] == True][::200]

,상호,시도,시군구,행정동,대분류,중분류,소분류,start_date,end_date,duration,...,담배소매,자동판매기,인허가목록,면적,종업원수,자본금,보증금,월세,소상공인_index,인허가_index
14,제일통증의학과의원,대구광역시,수성구,만촌2동,생활서비스,광고/인쇄,광고물제작,20040617,20190717,5508,...,False,False,"[(건강기능식품일반판매업, nan)]",NaN,NaN,NaN,NaN,NaN,19986225,"(58103,)"
1618,뽀끼마루,대구광역시,수성구,만촌2동,생활서비스,광고/인쇄,광고물제작,20100510,20190409,3256,...,False,False,"[(휴게음식점, 일반조리판매)]",NaN,NaN,NaN,NaN,NaN,19973378,"(159949,)"
3377,참존음식,대구광역시,수성구,만촌2동,생활서비스,광고/인쇄,광고물제작,20051216,20181106,4708,...,False,False,"[(일반음식점, 한식)]",NaN,NaN,NaN,NaN,NaN,20474256,"(107905,)"
5227,베스틴빌라,대구광역시,수성구,만촌2동,생활서비스,광고/인쇄,광고물제작,20140122,20181001,1713,...,False,False,"[(외국인관광도시민박업, nan)]",NaN,NaN,NaN,NaN,NaN,11868360,"(24275,)"
6545,스시이타,대구광역시,수성구,만촌2동,생활서비스,광고/인쇄,광고물제작,20171025,20200625,974,...,False,False,"[(일반음식점, 횟집)]",NaN,NaN,NaN,NaN,NaN,15977646,"(113626,)"
7626,네네치킨 상동점,대구광역시,수성구,만촌2동,생활서비스,광고/인쇄,광고물제작,20140624,20200303,2079,...,False,False,"[(일반음식점, 통닭(치킨))]",NaN,NaN,NaN,NaN,NaN,23365574,"(144556,)"
8975,삼익이용소,대구광역시,수성구,만촌2동,생활서비스,광고/인쇄,광고물제작,19770830,20200714,15659,...,False,False,"[(이용업, 일반이용업)]",NaN,NaN,NaN,NaN,NaN,23095124,"(51770,)"
10471,선미용실,대구광역시,수성구,만촌2동,생활서비스,광고/인쇄,광고물제작,19970110,20190103,8028,...,False,False,"[(미용업, 일반미용업)]",NaN,NaN,NaN,NaN,NaN,26239205,"(48427,)"
12267,돈돈 들안길점,대구광역시,수성구,만촌2동,생활서비스,광고/인쇄,광고물제작,20041104,20190125,5195,...,False,False,"[(일반음식점, 식육(숯불구이))]",NaN,NaN,NaN,NaN,NaN,12050196,"(143433,)"
13372,짱구게임장,대구광역시,수성구,만촌2동,생활서비스,광고/인쇄,광고물제작,20161007,20181004,727,...,False,False,"[(청소년게임제공업, nan)]",NaN,NaN,NaN,NaN,NaN,14198884,"(19629,)"


In [112]:
characters = set()

for name in df['사업장명'].fillna(''):
    name = name.replace(' ', '')
    characters.update(name)

In [113]:
characters

{'추',
 'ㅊ',
 '릿',
 '食',
 '환',
 '켜',
 '옐',
 '렙',
 '둘',
 '솥',
 '퍼',
 '적',
 '뎜',
 '티',
 'ㅇ',
 '냄',
 '땜',
 '잽',
 '多',
 '휴',
 '를',
 '근',
 '냅',
 '랫',
 '빨',
 '암',
 '읽',
 '郞',
 '링',
 '캉',
 '걷',
 '익',
 '美',
 '믄',
 '데',
 '딩',
 '첫',
 '탯',
 '곰',
 '行',
 '겠',
 '하',
 '곱',
 '얌',
 '쁜',
 '휠',
 '꽆',
 '범',
 '흑',
 '辣',
 '협',
 '혁',
 '·',
 '겸',
 '울',
 '心',
 '벧',
 '之',
 '닌',
 '충',
 '찡',
 '었',
 '앞',
 '띨',
 '퓨',
 '풀',
 '웬',
 '굴',
 '8',
 '펠',
 '숭',
 '딱',
 '좁',
 '독',
 '찰',
 '늘',
 '뜨',
 '뇌',
 '米',
 '빗',
 '벰',
 '랍',
 '롸',
 '쭌',
 '툼',
 '봐',
 '긱',
 '툴',
 '챕',
 '2',
 '탑',
 '윅',
 '粧',
 '돔',
 '빡',
 '텟',
 '말',
 '옻',
 '씨',
 '욤',
 '록',
 '걸',
 '뗄',
 '쌔',
 'Ё',
 '品',
 '멍',
 '르',
 '［',
 '생',
 '큐',
 '丹',
 '벅',
 '앤',
 '뾰',
 '떡',
 '쿠',
 '퓰',
 '결',
 '李',
 '졍',
 '렉',
 '찜',
 '窟',
 '와',
 'ㆍ',
 '플',
 '햅',
 '캘',
 '킬',
 '쿱',
 '內',
 '山',
 '젠',
 '씸',
 '모',
 '城',
 '검',
 '섹',
 '헬',
 '禮',
 '쇼',
 'B',
 '강',
 '야',
 '펫',
 '콩',
 '롬',
 '깎',
 '뻔',
 '빕',
 '읒',
 '륵',
 '솟',
 'k',
 'N',
 '탄',
 '春',
 '툰',
 '콜',
 '송',
 '딥',
 '후',
 '곤',
 '퓸',
 '첩',
 '꼭'

In [114]:
characters2 = set()

for name in smallbiz_df['상호명'].fillna(''):
    name = name.replace(' ', '').replace('·', '')
    characters2.update(name)

In [115]:
characters - characters2

{'"',
 '$',
 '<',
 '>',
 '@',
 '·',
 'α',
 'Ё',
 '₂',
 'Ⅳ',
 'Ⅴ',
 '→',
 'ⓝ',
 'し',
 'す',
 'タ',
 'ニ',
 'ネ',
 'ㄱ',
 'ㄷ',
 'ㅇ',
 'ㅊ',
 'ㅋ',
 'ㅡ',
 'ㅣ',
 'ㆍ',
 '一',
 '丁',
 '三',
 '中',
 '丹',
 '之',
 '乭',
 '事',
 '五',
 '亨',
 '作',
 '信',
 '元',
 '全',
 '公',
 '凱',
 '利',
 '化',
 '印',
 '厚',
 '名',
 '周',
 '味',
 '品',
 '商',
 '善',
 '國',
 '團',
 '土',
 '坊',
 '城',
 '壽',
 '夜',
 '好',
 '學',
 '山',
 '岡',
 '巢',
 '平',
 '幸',
 '庭',
 '心',
 '志',
 '情',
 '房',
 '手',
 '挺',
 '掛',
 '揚',
 '文',
 '新',
 '明',
 '星',
 '春',
 '有',
 '東',
 '林',
 '校',
 '森',
 '榮',
 '毛',
 '水',
 '池',
 '河',
 '流',
 '海',
 '淸',
 '湖',
 '源',
 '滴',
 '潭',
 '火',
 '燒',
 '珉',
 '生',
 '田',
 '男',
 '畔',
 '留',
 '發',
 '白',
 '眞',
 '知',
 '社',
 '稼',
 '空',
 '窟',
 '笑',
 '等',
 '米',
 '粧',
 '緣',
 '羊',
 '臺',
 '舜',
 '茶',
 '行',
 '西',
 '談',
 '豚',
 '軸',
 '通',
 '道',
 '酒',
 '釜',
 '門',
 '間',
 '際',
 '靑',
 '非',
 '高',
 '鮮',
 '鳳',
 '齋',
 '갖',
 '갠',
 '것',
 '겅',
 '깁',
 '깥',
 '껠',
 '껨',
 '꽆',
 '꾀',
 '꿒',
 '꿰',
 '낀',
 '낑',
 '낯',
 '넓',
 '넴',
 '넵',
 '놓',
 '뇸',
 '늦',
 '댐',
 '덥',
 '뎐',
 '뎜',
 '돗',
 '됨'

In [116]:
characters2 - characters

{'´',
 '’',
 '∥',
 '♥',
 '月',
 '香',
 '겡',
 '괘',
 '굶',
 '긍',
 '깽',
 '껌',
 '껜',
 '껴',
 '꽂',
 '꽝',
 '끊',
 '끓',
 '낼',
 '눗',
 '늙',
 '닦',
 '댑',
 '댜',
 '뎃',
 '돕',
 '돝',
 '뚫',
 '렸',
 '롄',
 '릎',
 '맡',
 '멉',
 '뫙',
 '묶',
 '뮨',
 '밉',
 '봤',
 '뵤',
 '빰',
 '뺄',
 '뽄',
 '뿅',
 '섀',
 '셍',
 '셴',
 '솨',
 '썩',
 '썽',
 '왜',
 '욋',
 '욥',
 '잗',
 '쟌',
 '죄',
 '줍',
 '쨍',
 '쩍',
 '쩔',
 '쫄',
 '챗',
 '챙',
 '챨',
 '췌',
 '칵',
 '캅',
 '톺',
 '툇',
 '툽',
 '틉',
 '펴',
 '펼',
 '푀',
 '헉',
 '헷',
 '횃'}

In [176]:
' '.join(sorted(characters | characters2))

'! " # $ % & \' ( ) * + , - . / 0 1 2 3 4 5 6 7 8 9 : ; < = > ? @ A B C D E F G H I J K L M N O P Q R S T U V W X Y Z [ ] _ ` a b c d e f g h i j k l m n o p q r s t u v w x y z ° ´ · α Ё ’ ₂ ℃ Ⅰ Ⅱ Ⅲ Ⅳ Ⅴ → ∥ ⓝ ♡ ♥ し す タ ニ ネ ㄱ ㄷ ㅇ ㅊ ㅋ ㅡ ㅣ ㆍ ㈜ 一 丁 三 中 丹 之 乭 事 五 亨 人 休 作 信 元 內 全 公 冒 凱 利 化 印 厚 名 周 味 品 商 善 國 團 土 坊 城 壽 多 夜 大 太 好 學 安 家 小 屋 山 岡 巢 平 幸 庭 心 志 情 愛 房 手 挺 掛 揚 文 新 明 星 春 月 有 東 林 校 森 椒 榮 正 母 毛 水 池 河 流 海 淸 湖 源 滴 潭 火 燒 珉 生 田 男 畔 留 發 白 眞 知 社 秀 稼 空 窟 笑 等 米 粧 緣 羊 美 臺 舜 花 茶 菜 行 西 談 豚 軸 辛 辣 通 道 郞 酒 金 釜 門 間 際 靑 非 食 香 高 鮮 鳳 齋 가 각 간 갈 감 갑 값 갓 강 갖 같 개 객 갠 갤 갬 갭 갯 갱 갸 걍 거 건 걷 걸 검 겁 것 겅 겉 게 겐 겔 겜 겟 겠 겡 겨 격 견 결 겸 겹 경 계 고 곡 곤 곧 골 곰 곱 곳 공 곶 과 곽 관 괄 괌 광 괘 괜 괴 교 구 국 군 굳 굴 굶 굼 굽 굿 궁 궈 권 궐 귀 귄 규 균 귤 그 극 근 글 금 급 긋 긍 기 긱 긴 길 김 깁 깃 깅 깊 까 깍 깎 깐 깔 깜 깡 깥 깨 깻 깽 꺄 꺼 꺽 꺾 껌 껍 껑 께 껜 껠 껨 껴 꼬 꼭 꼰 꼴 꼼 꼽 꽁 꽂 꽃 꽆 꽈 꽐 꽝 꾀 꾸 꾹 꾼 꿀 꿈 꿉 꿍 꿒 꿔 꿰 뀌 뀨 끄 끈 끊 끌 끓 끔 끗 끝 끼 끽 낀 낌 낑 나 낙 낚 난 날 낡 남 납 낫 났 낭 낮 낯 내 낵 낸 낼 냄 냅 냇 냉 냐 냠 냥 너 넉 넌 널 넓 넘 넙 넛 넝 네 넥 넨 넬 넴 넵 넷 녀 녁 년 념 녕 녘 녜 노 녹 논 놀 놈 놋 농 높 놓 뇌 뇨 뇸 뇽 누 눅 눈 눌 눔 눗 뉘 뉴 늄 느 늑 는 늘 늙 능 늦 늬 

In [127]:
special_characters = ['!', '"', '#', '$', '%', '&', '\'', '(', ')', '*', '+', ',', '-', '.', 
                      '/', ':', ';', '<', '=', '>', '?', '@', '[', ']', '_', '`', '°', '´', 
                      '·', 'α', 'Ё', '’', '₂', '℃', 'Ⅰ', 'Ⅱ', 'Ⅲ', 'Ⅳ', 'Ⅴ', '→', 
                      '∥', 'ⓝ', '♡', '♥', 'ㆍ', '㈜', '＆', '＇', '＋', '？', '［', '］', ' ', "'"]

name_list = []

for name in df['사업장명'].fillna(''):
    for char in special_characters:
        name = name.replace(char, '')
    
    name_list.append(name)

df['검색용_사업장명'] = name_list

In [128]:
name_list = []

for name in smallbiz_df['상호명'].fillna(''):
    for char in special_characters:
        name = name.replace(char, '')
    
    name_list.append(name)

smallbiz_df['검색용_상호명'] = name_list

In [129]:
df['검색용_사업장명']

0              광화문자생한방병원
2                 서울송도병원
3         의료법인제일의료재단제일병원
4             인제대학교서울백병원
5                국립중앙의료원
               ...      
251221       일반행정사박종진사무소
251222          우종일행정사무소
251224          조승수행정사무소
251228          효성행정사사무소
251230         문기태행정사사무소
Name: 검색용_사업장명, Length: 110572, dtype: object

In [130]:
smallbiz_df['검색용_상호명']

상가업소번호
19977290          피자마루
19973708            왕실
20013190          청암회관
20613530      은하노래광장주점
20622823         인덕기름집
               ...    
16512324         펫앤아쿠아
16457088           차수학
16467229         김호겸수학
16465047    극동1번지부동산중개
16476573        와이디자인탭
Name: 검색용_상호명, Length: 76413, dtype: object

In [132]:
indices = []

for i, (idx, row) in enumerate(smallbiz_df.iterrows()):
    result = df[df['검색용_사업장명'].str.contains(row['검색용_상호명'], regex=False) & 
                (df['도로명주소'].str.contains(row['도로명주소']))]

    if len(result):
        indices.append(tuple(result.index))
    else:
        indices.append(np.nan)
        
    if i % 1000 == 0:
        print(i, end='\r')

C:\Users\eyshi\Anaconda3\lib\site-packages\pandas\core\strings.py:1954: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)


In [133]:
np.sum(pd.notnull(indices))

18786

In [134]:
union_index = {}

for so_index, in_index in zip(smallbiz_df.index, indices):
    if pd.notnull(in_index):
        union_index[so_index] = in_index

In [135]:
with open('20200924_union_index.pkl', 'wb') as file:
    pickle.dump(file=file, obj=union_index)

In [169]:
lines = []

for so_index, in_indices in union_index.items():
    so_item = smallbiz_df.loc[so_index]
    in_items = [df.loc[in_index] for in_index in in_indices]
    
    if pd.notnull(so_item['지점명']):
        name = '%s %s' % (so_item['상호명'], so_item['지점명'])
    else:
        name = so_item['상호명']
    
    start_dates = np.array([float(item['인허가일']) for item in in_items])
    end_dates = np.array([item['end_date'] for item in in_items])
    
    if pd.isnull(start_dates).all():
        # 인허가일 없음 = 안 쓰는 데이터
        continue
    else:
        start_date = int(min(start_dates))
        in_item = in_items[np.argmin(start_dates)]
    
    if pd.isnull(end_dates).any():
        end_date = 20200831
        fail = False
        is_accurate_end_date = True
    else:
        end_date = int(max(end_dates))
        fail = True
        is_accurate_end_date = not in_items[np.argmax(end_dates)]['censored']
        
    duration = (datetime.datetime.strptime(str(end_date), '%Y%m%d') - datetime.datetime.strptime(str(start_date), '%Y%m%d')).days
    
    if in_item['영업상태'] == '휴업' or pd.notnull(in_item['데이터']['휴업시작일자']) or pd.notnull(in_item['데이터']['휴업종료일자']):
        have_been_rest = True
    else:
        have_been_rest = False
        
    in_categories = set([(in_item['업종']) for in_item in in_items])
    
    can_sell_internet = '통신판매업' in in_categories
    can_sell_can = '식품자동판매기업' in in_categories
    
    in_categories = [(in_item['업종'], in_item['상세업종']) for in_item in in_items]
    
    area = np.nan
    for area_name in area_list:
        if area_name in in_item:
            area = in_item['데이터'][area_name]
            break
    
    employee = np.nan
    for employee_name in employee_list:
        if employee_name in in_item:
            employee = in_item['데이터'][employee_name]
            break
        
    capital = in_item['데이터'].get('자본금', np.nan)
        
    line = [name, so_item['시도명'], so_item['시군구명'], so_item['행정동명'], 
            so_item['상권업종대분류명'], so_item['상권업종중분류명'], so_item['상권업종소분류명'], 
            start_date, end_date, duration, fail, so_item['층정보'],
            in_status[in_item['영업상태']], have_been_rest, is_accurate_end_date, 
            can_sell_internet, can_sell_can, in_categories,
            area, employee, capital, 
            so_index, in_indices]
    lines.append(line)

In [170]:
preproc_union_df = pd.DataFrame(
    lines, 
    columns=['상호', '시도', '시군구', '행정동', '대분류', '중분류', '소분류', 'start_date', 'end_date', 'duration', 'fail', '층정보', 
             '영업상태', 'have_been_휴업', 'is_accurate_end_date', '통신판매', '자동판매기', '인허가목록',
             '면적', '종업원수', '자본금', '소상공인_index', '인허가_index'])

In [171]:
preproc_union_df.to_pickle('외부데이터/지방행정 인허가 데이터/preproc_approval_roi_df.pkl')

In [172]:
preproc_union_df[preproc_union_df['fail'] == True][::200]

,상호,시도,시군구,행정동,대분류,중분류,소분류,start_date,end_date,duration,...,have_been_휴업,is_accurate_end_date,통신판매,자동판매기,인허가목록,면적,종업원수,자본금,소상공인_index,인허가_index
17,제일통증의학과의원,서울특별시,중구,약수동,의료,병원,기타병원,20040617,20190717,5508,...,False,True,False,False,"[(건강기능식품일반판매업, nan)]",NaN,NaN,NaN,19986225,"(58103,)"
1518,웰빙식당,서울특별시,노원구,중계4동,음식,한식,한식/백반/한정식,20040412,20200102,5743,...,False,True,False,False,"[(일반음식점, 분식)]",NaN,NaN,NaN,20695963,"(125022,)"
3229,그린푸드,서울특별시,노원구,상계2동,음식,일식/수산물,갈치/생선구이,20050805,20200518,5400,...,False,True,False,False,"[(휴게음식점, 백화점)]",NaN,NaN,NaN,20387561,"(163672,)"
5296,디어베이비 서울역롯데마트점,서울특별시,중구,회현동,소매,유아용품,유아용품판매,20150428,20190611,1505,...,False,True,False,False,"[(의료기기판매(임대)업, nan)]",NaN,NaN,NaN,4016836,"(10515,)"
6569,떡볶기날다,서울특별시,노원구,월계2동,음식,제과제빵떡케익,떡/한과집,20170412,20190722,831,...,False,True,False,False,"[(휴게음식점, 일반조리판매)]",NaN,NaN,NaN,12539838,"(165425,)"
7820,밥하우스,서울특별시,노원구,월계2동,음식,한식,한식/백반/한정식,20171113,20181015,336,...,False,True,False,False,"[(일반음식점, 한식)]",NaN,NaN,NaN,15992944,"(122244,)"
8930,한마음식당,대구광역시,수성구,범어1동,음식,유흥주점,호프/맥주,20010604,20200810,7007,...,False,True,False,False,"[(일반음식점, 호프/통닭)]",NaN,NaN,NaN,19921713,"(149536,)"
10287,디저트스트릿,대구광역시,중구,동인동,음식,제과제빵떡케익,제과점,20140507,20190121,1720,...,False,True,False,False,"[(제과점영업, 제과점영업)]",NaN,NaN,NaN,23174756,"(83317,)"
11842,세탁소,대구광역시,수성구,만촌1동,생활서비스,세탁/가사서비스,세탁소/빨래방,20030219,20200504,6284,...,False,True,False,False,"[(세탁업, 일반세탁업)]",NaN,NaN,NaN,23061011,"(53442,)"
13682,원스킨앤바디,대구광역시,수성구,수성1가동,생활서비스,이/미용/건강,비만/피부관리,20160411,20190607,1152,...,False,True,False,False,"[(미용업, 피부미용업)]",NaN,NaN,NaN,11924169,"(48876,)"


In [173]:
preproc_union_df.iloc[-1]

상호                                         라메르
시도                                       대구광역시
시군구                                         중구
행정동                                       성내1동
대분류                                         소매
중분류                                       의복의류
소분류                                   맞춤양장/양장점
start_date                            20200731
end_date                              20200831
duration                                    31
fail                                     False
층정보                                        NaN
영업상태                                         0
have_been_휴업                             False
is_accurate_end_date                      True
통신판매                                      True
자동판매기                                    False
인허가목록                   [(통신판매업, 의류/패션/잡화/뷰티)]
면적                                         NaN
종업원수                                       NaN
자본금                                        NaN
소상공인_index   